# 6.5 Final Modeling

##### Description

Pipelines to train the models are implemented in this notebook. The preprocessing steps of scaling and encoding are embedded in the pipeline. The exploratory models are trained to determine which has the highest accuracy score.

##### Notebook Steps

1. Connect Spark
1. Input train and test data
1. Input model parameters
1. Define model pipeline
1. Train models
1. Evaluate models

In [1]:
%load_ext sparkmagic.magics

In [2]:
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HTML(value='…

Added endpoint http://ec2-54-91-225-25.compute-1.amazonaws.com:8998/
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1612113777859_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [3]:
%%spark
spark.sparkContext.setCheckpointDir('./checkpoints')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 2. Load Training Data
There are two seperate datasets we will be working with while modeling. They are train.csv and validate.csv. Train is used to train the model, while validate is used to perform evaluation on unseen data. Both datasets are required for final modeling, but only the training dataset will be loaded right now.

In [4]:
%%spark
train = spark.read.csv("s3://jolfr-capstone3/training/train.csv", header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Define Pipeline Steps

 1. Feature Hasher
 1. Standard Scaler
 1. Models

In [5]:
%%spark
cols = train.drop("label").columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
%%spark
from pyspark.ml.feature import FeatureHasher
hasher = FeatureHasher(inputCols=cols, outputCol="hash")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
%%spark
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol=hasher.getOutputCol(), outputCol="features")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Load Tuned Parameters from S3

In [8]:
%%spark
import boto3
import pickle

s3 = boto3.client('s3')
## param keys are case sensitive, don't ask me why
obj = s3.get_object(Bucket = 'jolfr-capstone3', Key = 'LogisticParams')
serializedObj = obj['Body'].read()

lr_params = pickle.loads(serializedObj)
lr_params

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'aggregationDepth': 2, 'elasticNetParam': 0.4, 'family': 'auto', 'featuresCol': 'features', 'fitIntercept': True, 'labelCol': 'label', 'maxIter': 10, 'predictionCol': 'prediction', 'probabilityCol': 'probability', 'rawPredictionCol': 'rawPrediction', 'regParam': 0.01, 'standardization': True, 'threshold': 0.5, 'tol': 1e-06}

In [9]:
%%spark
import boto3
import pickle

s3 = boto3.client('s3')
obj = s3.get_object(Bucket = 'jolfr-capstone3', Key = 'ForestParams')
serializedObj = obj['Body'].read()

rf_params = pickle.loads(serializedObj)
rf_params

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'featuresCol': 'features', 'labelCol': 'label', 'predictionCol': 'prediction', 'probabilityCol': 'probability', 'rawPredictionCol': 'rawPrediction', 'maxDepth': 30, 'minInstancesPerNode': 50, 'numTrees': 100}

## Define Models

In [10]:
%%spark
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(**lr_params) ## Unpacks the dictionary

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
%%spark
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(**rf_params, cacheNodeIds=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Define Pipelines

In [12]:
%%spark
from pyspark.ml import Pipeline
lr_pipe = Pipeline(stages=[hasher, scaler, lr])
rf_pipe = Pipeline(stages=[hasher, scaler, rf])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Train Models

In [ ]:
%%spark
rf_model = rf_pipe.fit(train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
lr_model = lr_pipe.fit(train)

## Load Validation Data

In [ ]:
%%spark
validate = spark.read.csv("s3://jolfr-capstone3/validation/validate.csv", header=True, inferSchema=True)

## Evaluate Models

In [ ]:
%%spark
# Compute raw scores on the test set
predictionAndLabels = validate.rdd.map(lambda lp: (float(lr_model.predict(lp.features)), lp.label))

# Instantiate metrics object
lr_metrics = BinaryClassificationMetrics(predictionAndLabels)
lr_metrics

In [ ]:
%%spark
# Compute raw scores on the test set
predictionAndLabels = validate.rdd.map(lambda lp: (float(rf_model.predict(lp.features)), lp.label))

# Instantiate metrics object
rf_metrics = BinaryClassificationMetrics(predictionAndLabels)
rf_metrics

## Save Metrics to File

In [ ]:
%%spark
import boto3
import pickle

s3 = boto3.client('s3')

serialized = pickle.dumps(rf_metrics)

s3.put_object(Bucket='jolfr-capstone3',Key='rf-metrics', Body=serialized)

In [ ]:
%%spark
import boto3
import pickle

s3 = boto3.client('s3')

serialized = pickle.dumps(lr_metrics)

s3.put_object(Bucket='jolfr-capstone3',Key='lr-metrics', Body=serialized)